# Deep Convolutional Generative Adversarial Network (DCGAN)

In [1]:
# libraries
%pip install torch torchvision
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

     ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
     ---------------------------------------- 0.1/192.3 MB 1.7 MB/s eta 0:01:57
     ---------------------------------------- 0.2/192.3 MB 2.5 MB/s eta 0:01:18
     ---------------------------------------- 0.3/192.3 MB 2.9 MB/s eta 0:01:06
     ---------------------------------------- 0.6/192.3 MB 3.3 MB/s eta 0:00:58
     ---------------------------------------- 0.8/192.3 MB 3.8 MB/s eta 0:00:50
     ---------------------------------------- 1.4/192.3 MB 5.4 MB/s eta 0:00:36
      --------------------------------------- 2.7/192.3 MB 8.5 MB/s eta 0:00:23
      -------------------------------------- 4.2/192.3 MB 12.2 MB/s eta 0:00:16
     - ------------------------------------- 5.7/192.3 MB 14.1 MB/s eta 0:00:14
     - ------------------------------------- 7.5/192.3 MB 16.6 MB/s eta 0:00:12
     - ------------------------------------- 9.2/192.3 MB 18.3 MB/s eta 0:00:11
     -- ----------------------------------- 11.


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# hyperparameters
batchSize = 64
imageSize = 64
epochs = 15

In [14]:
# transformations
transform = transforms.Compose([
    transforms.Resize(imageSize),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [15]:
# dataset loader
dataset = dset.CIFAR10(
    root = 'C:/Users/b/Desktop/DCGAN/', 
    download = True, 
    transform = transform
    )
dataloader = torch.utils.data.DataLoader(
    dataset, 
    batch_size = batchSize, 
    shuffle = True, 
    num_workers = 2
    )

Files already downloaded and verified


In [16]:
# initialize weights
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [17]:
# define the generator
class G(nn.Module):
    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh() # -1, 1
        )

    def forward(self, input):
        output = self.main(input)
        return output

In [18]:
# create the generator
netG = G()
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [19]:
# define the discriminator
class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid() # 0, 1
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1)

In [20]:
# create the discriminator
netD = D()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [21]:
# train
criterion = nn.BCELoss() # measure error between prediction and real
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

for epoch in range(epochs):
    for i, data in enumerate(dataloader, 0):

        # update the weights of the discriminator
        netD.zero_grad() # initialize the gradients of the discriminator with 0

        # train the discriminator with a real image from dataset
        real, _ = data # get a real image
        input = Variable(real) # make it a variable
        target = Variable(torch.ones(input.size()[0])) # get the target.
        output = netD(input) # get a prediction (value between 0 and 1)
        errD_real = criterion(output, target) # compute the loss

        # train the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)) # input vector of the generator
        fake = netG(noise) # forward in the generator
        target = Variable(torch.zeros(input.size()[0])) # get the target
        output = netD(fake.detach()) # forward in the discriminator
        errD_fake = criterion(output, target) # compute the loss

        # backpropagate
        errD = errD_real + errD_fake # total error of the discriminator
        errD.backward() # backpropagate the loss error by computing the gradients
        optimizerD.step() # update the weights

        # update the weights of the generator
        netG.zero_grad() # initialize the gradients of the generator with 0
        target = Variable(torch.ones(input.size()[0])) # get the target
        output = netD(fake) # forward the discriminator
        errG = criterion(output, target) # compute the loss
        errG.backward() # backpropagate the loss by computing the gradients
        optimizerG.step() # update the weights

        # prints
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, epochs, i, len(dataloader), errD.item(), errG.item())) # losses of the discriminator and the generator
        if i % 100 == 0:
            vutils.save_image(real, 'C:/Users/b/Desktop/DCGAN/results/real_samples.png', normalize = True) # save real images
            fake = netG(noise) # get generated images
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("C:/Users/b/Desktop/DCGAN/results", epoch), normalize = True) # save generated images to minibatch

[0/15][0/782] Loss_D: 1.5579 Loss_G: 5.6435
[0/15][1/782] Loss_D: 1.0083 Loss_G: 6.6428
[0/15][2/782] Loss_D: 0.7964 Loss_G: 5.5590
[0/15][3/782] Loss_D: 1.2267 Loss_G: 6.9510
[0/15][4/782] Loss_D: 0.6533 Loss_G: 6.6867
[0/15][5/782] Loss_D: 1.2618 Loss_G: 8.0045
[0/15][6/782] Loss_D: 0.7098 Loss_G: 7.8708
[0/15][7/782] Loss_D: 1.1188 Loss_G: 9.4077
[0/15][8/782] Loss_D: 0.9037 Loss_G: 7.4573
[0/15][9/782] Loss_D: 1.0281 Loss_G: 9.3299
[0/15][10/782] Loss_D: 0.5844 Loss_G: 8.4017
[0/15][11/782] Loss_D: 0.5894 Loss_G: 9.9831
[0/15][12/782] Loss_D: 0.5285 Loss_G: 8.4917
[0/15][13/782] Loss_D: 0.6815 Loss_G: 12.7530
[0/15][14/782] Loss_D: 0.5258 Loss_G: 9.4658
[0/15][15/782] Loss_D: 0.7539 Loss_G: 11.9123
[0/15][16/782] Loss_D: 0.4919 Loss_G: 8.9566
[0/15][17/782] Loss_D: 0.7667 Loss_G: 13.0532
[0/15][18/782] Loss_D: 0.1835 Loss_G: 9.9946
[0/15][19/782] Loss_D: 0.6503 Loss_G: 13.1834
[0/15][20/782] Loss_D: 0.2925 Loss_G: 9.7395
[0/15][21/782] Loss_D: 1.1033 Loss_G: 15.5890
[0/15][22/782] 